# Linear Regression Forecasting Model

In [1]:
import tseriesRoutines as routines
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

In [2]:
# RESULT REPRODUCIBILITY
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(42)

In [3]:
def genData(mongoid, conn, cursor, impute=True, freq='daily'):
    '''
    Generate a timeseries dataframe for timeseries modelling.
    mongoid: str. string of mongodb id.
    conn: sqlite3 connection.
    cursor: sqlite3 cursor.
    impute:
    freq:
    actualrevcount:
    '''
    np.random.seed(42)
    initial = routines.sqlToDf(conn, cursor)
    allproduct = initial.selectReview3(mongoid, impute=impute)
    product = routines.tsSalesRateSentiment(allproduct, freq=freq)
    return product
    # product = genData('5aa2ad7735d6d34b0032a795', conn, c, impute=True, 
    #   freq='daily')

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
    n_vars = 1 if type(data) is list else data.shape[1]
    df = data.copy()
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg = agg.dropna()
    return agg

In [5]:
#Adj R square
def adj_r2_score(model,X_test, y_test,):
    y_pred = model.predict(X_test)
    # model.coefs_ doesn't exist
    adj = 1 - float(len(y_test)-1)/(len(y_test)-model.n_features_-1) * \
            (1 - r2_score(y_test,y_pred))
    return adj

# Evalute random search
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    MSE = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
#    r2_adjusted = adj_r2_score(model, X_test, y_test)
    print('Model Validation')
    print('MSE: {0}'.format(MSE))
    print('R^2: {0}'.format(r2))
#    print('R^2 Adjusted: {0}'.format(r2_adjusted))

In [6]:
def splitDataNN(df, n_in=1, n_out=1, scale=True, percent=0.2):
    '''
    df: pandas dataframe. 3 columns (sales, rating, ovsentiment) with date as index
    n_in:
    n_out:
    scale:
    percent:
    X_train, y_train, X_test, y_test, dftrain = splitDataNN(product, n_in=1, 
        n_out=1, scale=True, percent=0.2)
    '''
    dftrain = series_to_supervised(df, n_in=n_in, n_out=n_out)
    # specific to this case
    dftrain = dftrain.drop(dftrain.columns[[4, 5]], axis=1)
    values = dftrain.values

    if scale:
        scaler = StandardScaler()
        values = scaler.fit_transform(values)
    else:
        pass

    # training data
    X, y = values[:, :-1], values[:, -1]
    # train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=percent, 
            shuffle=False, random_state=42)
    return X_train, y_train, X_test, y_test, dftrain, scaler

In [7]:
def evalForecast(model, X, y, inverse=False, scaler=None):
    '''
    Evaluate time series forecasting model
    '''
    if inverse and scaler:
        # make prediction
        ypred = model.predict(X)
        # invert scaling predicted data
        inv_ypred = np.concatenate((X[:, :], ypred.reshape((-1,1))), axis=1)
        inv_ypred = scaler.inverse_transform(inv_ypred)
        inv_ypred = inv_ypred[:, -1]
        # invert scaling for actual data
        y = y.reshape((len(y), 1))
        inv_y = np.concatenate((X[:, :], y.reshape((-1,1))), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:, -1]
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_pred=inv_ypred, y_true=inv_y))
        # MAE
        mae = mean_absolute_error(y_pred=inv_ypred, y_true=inv_y)
    else:
        # make prediction
        ypred = model.model.predict(X)
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_pred=ypred, y_true=y))
        # MAE
        mae = mean_absolute_error(y_pred=ypred, y_true=y)

    print('Validasi RMSE: {0:.5f}'.format(rmse))
    print('Validasi MAE: {0:.5f}'.format(mae))

In [8]:
# make connection to sqlite db
conn = sqlite3.connect('product.db')
c = conn.cursor()

# enable foreign keys
c.execute("PRAGMA foreign_keys = ON")
conn.commit()

<br>pilihan:
>     2 data di database product.db dgn review > 900:
>         5aa2ad7735d6d34b0032a795
>         5aa39533ae1f941be7165ecd
>     cluster 3
>         5a93e8768cbad97881597597
>         or 
>         5a95d7ae35d6d33d3fea56ff
>     cluster 1
>         5aa2c35e35d6d34b0032a796
>     cluster 2 
>         5a92474635d6d32207bcd343
</br>

## <font color=blue> 1. Mongodb ID: 5aa2ad7735d6d34b0032a795 </font>

In [8]:
product = genData('5aa2ad7735d6d34b0032a795', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [9]:
product.shape

(479, 3)

In [10]:
product['Sales'].sum()

41548

In [11]:
product.head()

,Sales,rating,ovsentiment
date,,,
2016-11-06,82,3.500000,0.500000
2016-11-07,88,5.000000,1.000000
2016-11-08,74,0.000000,0.000000
2016-11-09,72,0.000000,0.000000
2016-11-10,98,4.666667,0.333333


In [12]:
X_train.shape

(382, 3)

In [8]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()

In [14]:
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
evaluate(slr, X_test, y_test)

Model Validation
MSE: 1.0625613715252527
R^2: -0.023007491415257597


In [16]:
y_pred = slr.predict(X_test)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0308061755370175

In [17]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8307621628305174

In [9]:
# POLYNOMIAL REGRESSION
###########################
from sklearn.preprocessing import PolynomialFeatures
regr = LinearRegression()
quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)

In [19]:
X_train_quad = quadratic.fit_transform(X_train)
X_train_cubic = cubic.fit_transform(X_train)
X_test_quad = quadratic.fit_transform(X_test)
X_test_cubic = cubic.fit_transform(X_test)

In [20]:
# quadratic
regr = regr.fit(X_train_quad, y_train)

In [21]:
evaluate(regr, X_test_quad, y_test)

Model Validation
MSE: 1.1190428923998945
R^2: -0.07738648591827091


In [22]:
y_pred = regr.predict(X_test_quad)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0578482369413367

In [23]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8451088115561755

In [24]:
# cubic
regr = regr.fit(X_train_cubic, y_train)

In [25]:
y_pred = regr.predict(X_test_cubic)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0407992278813654

In [26]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8354795394765859

## <font color=blue> 2. Mongodb ID: 5aa39533ae1f941be7165ecd </font>

In [27]:
product = genData('5aa39533ae1f941be7165ecd', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [28]:
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [29]:
evaluate(slr, X_test, y_test)

Model Validation
MSE: 1.1860735338073864
R^2: 0.006855414766489765


In [30]:
y_pred = slr.predict(X_test)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0890700316358846

In [31]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8410492013956025

In [32]:
regr = LinearRegression()
quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)

In [33]:
X_train_quad = quadratic.fit_transform(X_train)
X_train_cubic = cubic.fit_transform(X_train)
X_test_quad = quadratic.fit_transform(X_test)
X_test_cubic = cubic.fit_transform(X_test)

In [34]:
# quadratic
regr = regr.fit(X_train_quad, y_train)

In [35]:
evaluate(regr, X_test_quad, y_test)

Model Validation
MSE: 1.2574847555240893
R^2: -0.05293992350835475


In [36]:
y_pred = regr.predict(X_test_quad)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.121376277403838

In [37]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8684426792408699

In [38]:
# cubic
regr = regr.fit(X_train_cubic, y_train)

In [39]:
y_pred = regr.predict(X_test_cubic)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.4898142161370418

In [40]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.9484523286665023

## <font color=blue> 3. Mongodb ID: 5a93e8768cbad97881597597 </font>

In [41]:
product = genData('5a93e8768cbad97881597597', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [42]:
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [43]:
evaluate(slr, X_test, y_test)

Model Validation
MSE: 1.0790735990572746
R^2: -0.00880186883541878


In [44]:
y_pred = slr.predict(X_test)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0387846740577542

In [45]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8366687466529

In [46]:
regr = LinearRegression()
quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)

In [47]:
X_train_quad = quadratic.fit_transform(X_train)
X_train_cubic = cubic.fit_transform(X_train)
X_test_quad = quadratic.fit_transform(X_test)
X_test_cubic = cubic.fit_transform(X_test)

In [48]:
# quadratic
regr = regr.fit(X_train_quad, y_train)

In [49]:
evaluate(regr, X_test_quad, y_test)

Model Validation
MSE: 1.0762114581821525
R^2: -0.006126117092241978


In [50]:
y_pred = regr.predict(X_test_quad)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0374061201777018

In [51]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.840553366207255

In [52]:
# cubic
regr = regr.fit(X_train_cubic, y_train)

In [53]:
y_pred = regr.predict(X_test_cubic)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.0362859463072278

In [54]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.8355312105700679

## <font color=blue> 5. Mongodb ID: 5aa2c35e35d6d34b0032a796 </font>

In [69]:
product = genData('5aa2c35e35d6d34b0032a796', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [70]:
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [71]:
evaluate(slr, X_test, y_test)

Model Validation
MSE: 1.61718036004619
R^2: 0.01138448531197278


In [72]:
y_pred = slr.predict(X_test)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.2716840645562049

In [73]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.33147496352025696

In [74]:
regr = LinearRegression()
quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)

In [75]:
X_train_quad = quadratic.fit_transform(X_train)
X_train_cubic = cubic.fit_transform(X_train)
X_test_quad = quadratic.fit_transform(X_test)
X_test_cubic = cubic.fit_transform(X_test)

In [76]:
# quadratic
regr = regr.fit(X_train_quad, y_train)

In [77]:
evaluate(regr, X_test_quad, y_test)

Model Validation
MSE: 1.6565134808303148
R^2: -0.012660657956507837


In [78]:
y_pred = regr.predict(X_test_quad)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.2870561296347238

In [79]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.34248574384825864

In [80]:
# cubic
regr = regr.fit(X_train_cubic, y_train)

In [81]:
y_pred = regr.predict(X_test_cubic)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

1.3154266069559164

In [82]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

0.35463042867713773

## <font color=blue> 7. Mongodb ID: 5a9347b98cbad97074cb1890 </font>

In [13]:
product = genData('5a9347b98cbad97074cb1890', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [14]:
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
evaluate(slr, X_train, y_train)

Model Validation
MSE: 0.9694820025711656
R^2: 0.023861396592282702


In [16]:
y_pred = slr.predict(X_train)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_train.reshape(-1,1)))

0.9846227717106514

In [17]:
mean_absolute_error(y_pred=y_pred, y_true=y_train.reshape(-1,1))

0.7073056178537338

In [18]:
regr = LinearRegression()
quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)

In [19]:
X_train_quad = quadratic.fit_transform(X_train)
X_train_cubic = cubic.fit_transform(X_train)
X_test_quad = quadratic.fit_transform(X_test)
X_test_cubic = cubic.fit_transform(X_test)

In [20]:
# quadratic
regr = regr.fit(X_train_quad, y_train)

In [ ]:
evaluate(regr, X_test_quad, y_test)

In [ ]:
y_pred = regr.predict(X_test_quad)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

In [ ]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

In [ ]:
# cubic
regr = regr.fit(X_train_cubic, y_train)

In [ ]:
y_pred = regr.predict(X_test_cubic)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test.reshape(-1,1)))

In [ ]:
mean_absolute_error(y_pred=y_pred, y_true=y_test.reshape(-1,1))

In [21]:
evaluate(slr, X_test, y_test)

Model Validation
MSE: 1.0543450280396718
R^2: -0.027600666352811887


## performansi pada train set

In [9]:
from sklearn.linear_model import LinearRegression

In [30]:
product = genData('5a9347b98cbad97074cb1890', conn, c, impute=False, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [31]:
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [32]:
evaluate(slr, X_train, y_train)

Model Validation
MSE: 0.5744981747895025
R^2: 0.031684148889806196


In [33]:
y_pred = slr.predict(X_train)

In [34]:
evalForecast(slr, X_train, y_train, inverse=True, scaler=scaler)

Validasi RMSE: 0.26816
Validasi MAE: 0.13051
